In [15]:
import pandas as pd
import os

In [16]:
# experiment_type_A = 'reward-network-iii-8A-v1'
# experiment_type_B = 'reward-network-iii-8B-v1'


experiment_type_B = 'reward-network-iii-pilot-10B-v4'
experiment_type_A = 'reward-network-iii-pilot-9D-v4'

# Estimate the social learning rate

The social learning rate is the probability that and individual exposed to a optimal behavior will adopt it.

In [22]:
moves_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type_B,'moves.csv'))

moves_df = moves_df[~moves_df['ai_player']]


rel_trial_types = ['demonstration']
# moves_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)]

opt_df = moves_df.groupby(['session_id', 'trial_id', 'trial_type'])[['optimal','myopic']].sum().reset_index()
opt_df['all_optimal'] = opt_df['optimal'] == 10
opt_df['all_myopic'] = opt_df['myopic'] == 10
opt_df['other'] = ~(opt_df['all_optimal'] | opt_df['all_myopic'])


opt_df2 = opt_df[opt_df['trial_type'].isin(rel_trial_types)].copy()
opt_df2['all_optimal_discovered'] = opt_df2.groupby('session_id')['all_optimal'].transform('cummax')
opt_df2['not_discovered'] = ~opt_df2['all_optimal_discovered']

opt_player_df = opt_df2.groupby('session_id')['not_discovered'].sum().reset_index()
print(1 - opt_player_df.shape[0] / opt_player_df['not_discovered'].sum())

0.4545454545454546


In [23]:
opt_df.groupby(['trial_type'])[['all_optimal', 'all_myopic', 'other']].mean()

,all_optimal,all_myopic,other
trial_type,,,
demonstration,0.527778,0.125000,0.347222
individual,0.000000,0.277778,0.722222
observation,0.888889,0.111111,0.000000
repeat,0.777778,0.111111,0.111111
try_yourself,0.527778,0.097222,0.375000


In [33]:
df = opt_df.groupby(['session_id', 'trial_type'])[['all_optimal', 'all_myopic', 'other']].sum().reset_index()
df = df.groupby('trial_type')['all_optimal'].value_counts(normalize=True).unstack().fillna(0)
df

all_optimal,0,1,2,3,4
trial_type,,,,,
demonstration,0.444444,0.000000,0.000000,0.111111,0.444444
individual,1.000000,0.000000,0.000000,0.000000,0.000000
observation,0.111111,0.000000,0.000000,0.000000,0.888889
repeat,0.111111,0.055556,0.000000,0.277778,0.555556
try_yourself,0.222222,0.111111,0.166667,0.333333,0.166667


# Estimate the individual learning rate

The individual learning rate is the probability that and individual will adopt the optimal behavior on its own.

In [19]:
moves_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type_A,'moves.csv'))
rel_trial_types = ['individual', 'demonstration']

# moves_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)]

opt_df = moves_df.groupby(['session_id', 'trial_id','trial_type'])[['optimal','myopic']].sum().reset_index()
opt_df['all_optimal'] = opt_df['optimal'] == 10
opt_df['all_myopic'] = opt_df['myopic'] == 10
opt_df['other'] = ~(opt_df['all_optimal'] | opt_df['all_myopic'])

(opt_df['all_optimal'].sum() + 1) / (opt_df.shape[0] + 1)

0.0013698630136986301

In [20]:
opt_df

,session_id,trial_id,trial_type,optimal,myopic,all_optimal,all_myopic,other
0,65ba9808fa589ac7c2fb001e,4,individual,0,10,False,True,False
1,65ba9808fa589ac7c2fb001e,5,individual,0,8,False,False,True
2,65ba9808fa589ac7c2fb001e,8,individual,3,8,False,False,True
3,65ba9808fa589ac7c2fb001e,9,individual,0,6,False,False,True
4,65ba9808fa589ac7c2fb001e,10,individual,0,6,False,False,True
...,...,...,...,...,...,...,...,...
724,65bb81d4fa589ac7c2fb0093,21,try_yourself,0,10,False,True,False
725,65bb81d4fa589ac7c2fb0093,24,demonstration,2,10,False,True,False
726,65bb81d4fa589ac7c2fb0093,25,demonstration,0,10,False,True,False
727,65bb81d4fa589ac7c2fb0093,26,demonstration,0,10,False,True,False


In [21]:
opt_df.groupby(['trial_type'])[['all_optimal', 'all_myopic', 'other']].mean()

,all_optimal,all_myopic,other
trial_type,,,
demonstration,0.0,0.518868,0.481132
individual,0.0,0.317972,0.682028
observation,0.0,0.470000,0.530000
repeat,0.0,0.450000,0.550000
try_yourself,0.0,0.590000,0.410000
